# 03 - Load sql tables into Amazon Aurora PostgreSQL DB

In [2]:
%pip install -q flake8 2> /dev/null

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import os

import boto3

In [4]:
ssm = boto3.client("ssm")
secretsmanager = boto3.client("secretsmanager")
region = boto3.session.Session().region_name

In [5]:
security_group_parameter = "/AgenticLLMAssistant/SMProcessingJobSecurityGroupId"
dbsecret_arn_parameter = "/AgenticLLMAssistant/DBSecretARN"
subnet_ids_parameter = "/AgenticLLMAssistant/SubnetIds"
s3_bucket_name_parameter = "/AgenticLLMAssistant/AgentDataBucketParameter"

security_group = ssm.get_parameter(Name=security_group_parameter)
security_group = security_group["Parameter"]["Value"]

db_secret_arn = ssm.get_parameter(Name=dbsecret_arn_parameter)
db_secret_arn = db_secret_arn["Parameter"]["Value"]

subnet_ids = ssm.get_parameter(Name=subnet_ids_parameter)
private_subnets_with_egress_ids = json.loads(subnet_ids["Parameter"]["Value"])

s3_bucket_name = ssm.get_parameter(Name=s3_bucket_name_parameter)
s3_bucket_name = s3_bucket_name["Parameter"]["Value"]

In [6]:
sql_tables_s3_key = "structured_metadata"

In [7]:
!mkdir -p scripts

In [8]:
%%writefile scripts/load_sql_tables.py
import json
import os

import boto3
import dask.dataframe as dd
import psycopg2
import sqlalchemy

secretsmanager = boto3.client("secretsmanager")

secret_response = secretsmanager.get_secret_value(
    SecretId=os.environ["SQL_DB_SECRET_ID"]
)

database_secrets = json.loads(secret_response["SecretString"])

# Extract credentials
host = database_secrets['host']
dbname = database_secrets['dbname']
username = database_secrets['username']
password = database_secrets['password']
port = database_secrets["port"]

db_connection = psycopg2.connect(
    host=host,
    port=port,
    database=dbname,
    user=username,
    password=password,
)


def activate_vector_extension(db_connection):
    """Activate PGVector extension."""

    db_connection.autocommit = True
    cursor = db_connection.cursor()
    # install pgvector
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    db_connection.close()


def test_db_connection():
    # Connect to the database
    conn = psycopg2.connect(
        host=host,
        database=dbname,
        user=username,
        password=password
    )
    # Get cursor
    cur = conn.cursor()

    # Query to get all tables
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")

    # Fetch all the tables
    tables = cur.fetchall()

    # Print the table names
    print(f"SQL tables: {tables}")

    # Close connection
    conn.close()


def load_sql_tables(raw_tables_base_path, raw_tables_data_paths, columns_to_load, engine):
    """Load csv files as SQL tables into an Amazon Aurora PostgreSQL DB.

    Note:
        raw_tables_data_paths (List, str): a list of strings, each string
        can be a csv file, or a folder that contains a partitioned csv file.
    """

    for raw_table_path in raw_tables_data_paths:
        data_loading_path = os.path.join(
            raw_tables_base_path,
            raw_table_path
        )

        if os.path.isdir(data_loading_path):
            data_loading_path = os.path.join(data_loading_path, "*")
            table_name = raw_table_path
        else:
            table_name = raw_table_path.split(".")[0]

        print(f"Loading {table_name} data into a pandas dataframe")
        current_data_df = dd.read_csv(data_loading_path).compute()
        if columns_to_load == "all":
            columns_to_load = current_data_df.columns

        current_data_df = current_data_df[columns_to_load]

        current_data_df.to_sql(
            table_name, engine, if_exists='replace', index=False
        )

    return True


if __name__ == "__main__":
    test_db_connection()

    url_object = sqlalchemy.URL.create(
        "postgresql+psycopg2",
        username=username,
        password=password,
        host=host,
        database=dbname,
    )

    db_engine = sqlalchemy.create_engine(url_object)

    input_data_base_path = "/opt/ml/processing/input/"
    raw_sql_tables_base_path = os.path.join(input_data_base_path, "sqltables")
    tables_raw_data_paths = os.listdir(raw_sql_tables_base_path)
    columns_to_load = "all"

    print(raw_sql_tables_base_path, tables_raw_data_paths)
    load_sql_tables(
        raw_sql_tables_base_path,
        tables_raw_data_paths,
        columns_to_load,
        db_engine
    )

    test_db_connection()


Writing scripts/load_sql_tables.py


In [9]:
!flake8 --ignore=E501 scripts/load_sql_tables.py

## Attempt the same in a SageMaker processing job with VPC network config

In [10]:
from sagemaker.network import NetworkConfig

# Note if you enable network isolation, with enable_network_isolation=True
# the pip installation of the dependencies
# under scripts/requirements.txt won't work.
current_network_config = NetworkConfig(
    subnets=private_subnets_with_egress_ids, security_group_ids=[security_group]
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [11]:
%%time
from sagemaker.xgboost import XGBoostProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

# Initialize the XGBoostProcessor
xgb = XGBoostProcessor(
    framework_version="1.7-1",
    role=get_execution_role(),
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="frameworkprocessor-XGB",
    env={"SQL_DB_SECRET_ID": db_secret_arn, "AWS_DEFAULT_REGION": region},
    network_config=current_network_config,
)

# Run the processing job
xgb.run(
    code="load_sql_tables.py",
    source_dir="scripts",
    inputs=[
        ProcessingInput(
            input_name="sqltables",
            source=f"s3://{s3_bucket_name}/{sql_tables_s3_key}",
            destination="/opt/ml/processing/input/sqltables",
        )
    ],
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating processing-job with name frameworkprocessor-XGB-2023-10-02-14-11-56-009


Using provided s3_resource
  Obtaining dependency information for pandas==2.0.3 from https://files.pythonhosted.org/packages/f8/7f/5b047effafbdd34e52c9e2d7e44f729a0655efafb22198c45cf692cdc157/pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for dask==2023.5.0 from https://files.pythonhosted.org/packages/07/93/32d3e317fec6d0fc130284f922ad9bd13d9ae0d52245e6ff6e57647e924c/dask-2023.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/0f/ff/df058f6904693e2a76da8809e89f3b4ca23fac9929de7b385184f4c161ed/psycopg2_binary-2.9.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for SQLAlchemy from https://files.pythonhosted.org/packages/7c/70/5a36e9dd9bb62dca4d07d811c993eadd432769f541ea91c851f41f9d5ee8/SQLAlchemy-2.0.21-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency